In [ ]:
import tensorflow as tf
from process_data import *
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import pstats
import cProfile

In [ ]:
with cProfile.Profile() as pr:
    filepaths = grab_and_split_data (
        "/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/data/video/train",
        "/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/data/video/validate"
    )
    trainX, trainY = view_tf_records(filepaths["train_files"], 300000)
    validX, validY = view_tf_records(filepaths["test_files"], 300000)
    # validX, validY = view_tf_records(filepaths["validation_files"], 300000)
    # view_tf_records(filepaths["train_files"], 100)


    print(trainX.shape, trainY.shape)
    # print(testX.shape, testY.shape)
    # print(validX.shape, validY.shape)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
# stats.print_stats()
stats.dump_stats(filename='needs_profiling.prof')


In [ ]:
print(trainX.shape)
print(trainY.shape)
print(validX.shape)
print(validY.shape)


In [ ]:
import csv

def convert_csv_to_dict(file_path: str) -> dict:
    values = {}
    with open(file_path) as csv_file:
        csv_file = csv.reader(csv_file)
        for i, row in enumerate(csv_file):
            if i > 0:
                values[int(row[0])] = row[3]
    
    return values

            
str_labels = convert_csv_to_dict("/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/vocab.csv")

In [ ]:
import random

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(300, 300))
    for n in range(len(image_batch)):
        ax = plt.subplot(5,20, n+1)
        plt.imshow(image_batch[n])
        labels = [label for label, x in enumerate(label_batch[n]) if x == 1]
        title_str = ''
        for label in labels:
            title_str += str_labels[label] + ",\n"
        # print(title_str)
        plt.title(title_str, fontsize=80)
        plt.axis("off")

# show_batch(trainX, trainY)

# print(trainX.shape, trainY.shape)



In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(2, 2), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3862, activation='sigmoid'))

# model.summary()

optimizer=tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
results = model.fit(trainX, trainY, epochs=20, validation_data=(validX, validY), batch_size=32)

print(results.history)

In [ ]:
start_index = 0
end_index = 8
show_batch(validX[start_index:end_index], validY[start_index:end_index])
# trainX[0:2].shape
# model.predict(trainX[0:2])[0]
# model.predict(trainX[0:2])[1]

for index, model_predictions in enumerate(model.predict(validX[start_index:end_index], 1)):
    print(f"Currently on prediction {index}")
    for tuple in sorted(enumerate(model_predictions, 1), key=lambda x: x[1], reverse=True)[0:2]:
        label, confidence = tuple
        print(str_labels[label], confidence)
    print('\n')

# for tuple in sorted(enumerate(model.predict(trainX[0:2]), 1), key=lambda x: x[1], reverse=True)[0:2]:
#     label, confidence = tuple
#     print(str_labels[label], confidence)